In [2]:
#!pip3 install pandas
#!pip3 install PyArrow
from pyspark.sql.functions import col
import os

In [3]:
#df = pd.read_csv("train.gz")

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

In [5]:
from IPython.display import Javascript, HTML
import os
import time
import json
import requests
import xml.etree.ElementTree as ET
import datetime
import subprocess

In [6]:
if os.path.exists("/etc/hadoop/conf/hive-site.xml"):
    tree = ET.parse("/etc/hadoop/conf/hive-site.xml")
    root = tree.getroot()
    for prop in root.findall("property"):
        if prop.find("name").text == "hive.metastore.warehouse.dir":
            storage = (
                prop.find("value").text.split("/")[0]
                + "//"
                + prop.find("value").text.split("/")[2]
            )

In [7]:
os.environ["STORAGE"] = storage

In [8]:
spark = SparkSession\
    .builder\
    .appName("PythonSQL")\
    .config("spark.hadoop.fs.s3a.s3guard.ddb.region","us-east-2")\
    .config("spark.yarn.access.hadoopFileSystems",os.environ["STORAGE"])\
    .config("spark.rpc.message.maxSize", "1024")\
    .config("spark.dynamicAllocation.enabled", "true")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true")\
    .config("spark.driver.cores", 4)\
    .config("spark.driver.memory", "8g")\
    .getOrCreate()

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco


In [9]:
df = spark.read.parquet(os.environ["STORAGE"]+'/cde-workshop/clickthrough/data_large_from_csv',   
    header=True,
    sep=',',
    nullValue='NA')

In [10]:
df.take(2)

[Row(id='8.557257391414261e+18', click='0', hour='2014-10-24 07:00:00', C1='1005', banner_pos='1', site_id='e151e245', site_domain='7e091613', site_category='f028772b', app_id='ecad2386', app_domain='7801e8d9', app_category='07d7df22', device_id='a99f214a', device_ip='f73cf001', device_model='a0f5f879', device_type='1', device_conn_type='0', C14='4687', C15='320', C16='50', C17='423', C18='2', C19='39', C20='100148', C21='32'),
 Row(id='3.955540169721846e+18', click='1', hour='2014-10-24 09:00:00', C1='1005', banner_pos='0', site_id='1fbe01fe', site_domain='f3845767', site_category='28905ebd', app_id='ecad2386', app_domain='7801e8d9', app_category='07d7df22', device_id='a99f214a', device_ip='aa92a552', device_model='f07e20f8', device_type='1', device_conn_type='2', C14='20108', C15='320', C16='50', C17='2299', C18='2', C19='1319', C20='100084', C21='52')]

In [11]:
df.rdd.getNumPartitions()

25

In [12]:
df.dtypes

[('id', 'string'),
 ('click', 'string'),
 ('hour', 'string'),
 ('C1', 'string'),
 ('banner_pos', 'string'),
 ('site_id', 'string'),
 ('site_domain', 'string'),
 ('site_category', 'string'),
 ('app_id', 'string'),
 ('app_domain', 'string'),
 ('app_category', 'string'),
 ('device_id', 'string'),
 ('device_ip', 'string'),
 ('device_model', 'string'),
 ('device_type', 'string'),
 ('device_conn_type', 'string'),
 ('C14', 'string'),
 ('C15', 'string'),
 ('C16', 'string'),
 ('C17', 'string'),
 ('C18', 'string'),
 ('C19', 'string'),
 ('C20', 'string'),
 ('C21', 'string')]

In [13]:
#spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
df = df.withColumn("converted_date",df['hour'].cast(TimestampType()))

NameError: name 'TimestampType' is not defined

In [ ]:
df = df.withColumn('year',F.year("converted_date"))\
            .withColumn('month', F.month("converted_date"))\
            .withColumn('day', F.dayofmonth("converted_date"))\
            .withColumn('dt_hour', F.hour("converted_date"))

In [ ]:
df.dtypes

In [ ]:
#sparkDF.write\
# .option("header","true")\
# .parquet("s3a://go01-demo/cde-workshop/clickthrough")